Demo code for PINNs \
Author: Changrui Fang from Wuhan University\
2021300002027@whu.edu.cn

# 1. Base definition -- a DNN

In [61]:
# import the necessary package
import torch
import torch.nn as nn
import matplotlib as plt
import numpy as np
from collections import OrderedDict

In [62]:
# define a DNN 

class DNN(nn.Module):
    def __init__(self, layers):
        super().__init__()  # initialize the DNN
        
        # next define the construction of DNN
        
        self.DEP = len(layers) - 1  # the depth of the network is defined as number of all layers - 1
        self.ACT = torch.nn.Tanh  # the activate function of the network we use is Tanh
        self.IN = torch.nn.Linear(num_IN, num_Hidden)  # input layer
        
        layer_list = list()  # difine a empty list firstly
        
        # next define DNN except last layer
        for i in range(self.DEP - 1):
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layer[i], layer[i+1]))
            )
            layer_list.append(
            ('activation_%d' % i, self.ACT())
            )
            
        # define the last layer
        layer_list.append(
            ('layer_%d' % (self.DEP -1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        
        layerDict = OrderedDict(layer_list)
        
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, inputs):
        outputs = self.layers(inputs)
        return outputs

# 2. Specific examples

## 2.1 1D poisson problem

In [2]:
# define physis-informed neural network

class PINN():
    def __init__(self, X, u, layers, lb, ub):
        '''
        X: Input data
        layers: Structure of a neural network
        lb: low boundary condition
        ub: up boundary condition
        '''

        # define boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        
        # data
        self.x = torch.tensor(X, requires_grad=True).float().to(device)
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        
        # optimizer: Adam
        self.Adam = torch.optim.Adam(self.dnn.parameters())
        
        self.LBFGS = torch.optim.LBFGS(
            self.dnn.parameters(),  # 优化的模型参数为神经网络中的权重和偏置
            lr=1,  # 学习率
            max_iter=50000,  # L-BFGS算法的最大迭代次数
            max_eval=50000,  # 允许评估目标函数的最大次数
            history_size=50,  # L-BFGS算法历史缓存的大小

            # 停机准则
            tolerance_grad=1e-8,  # 梯度收敛容差，当梯度的L2范数小于此值时，算法停止
            tolerance_change=10 * np.finfo(float).eps,  # 参数变化的容差，当参数的变化小于此值时，算法停止
            
            line_search_fn="strong_wolfe"       # 线搜索准则： 强 wolfe 准则
        )
        
        self.iter = 0  # itration
        
    def net_u(self, x):
        u = self.dnn(x)
        return u
    
    def net_f(self, x):
        u = self.net_u(x)
        
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]

        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        f = np.pi**2*torch.sin(np.pi*x) - u_xx  # poisson equation: u'' = pi^2 * sin(pi*x)
        
        return f
    
    def DataLoss(self):  # data loss

        u_pre = self.net_u(self.x)
        loss = torch.mean(torch.square(u_pre - self.u))
        return loss
    
    def EqLoss(self):  # equation loss, s.t. physics information
        
        f_pre = self.net_f(self.x)
        loss = torch.mean(torch.square(f_pre))
        return loss
        
    def Loss(self):
        
        loss = self.DataLoss() + self.EqLoss()
        
        self.optimizer.zero_grad()  # clear the gradient
        loss.backward()  # backpropagation
        
        self.iter += 1
        if (self.iter + 1) % 100 == 0:
            print(
                'iteration：%.1f, Loss: %e, a: %.5f' %
                (
                    self.iter+1,
                    loss.item()
                )
            )
        return loss
    
    # next define the train part
    def train(self, num_epoch):
        self.dnn.train()
        
        for epoch in range(num_epoch):
            u_pred = self.net_u(self.x, self.t)
            f_pred = self.net_f(self.x, self.t)
            
            loss = self.Loss()
            
            self.optimizer_Adam.zero_grad()
            loss.backward
            self.optimizer_Adam.step()
            
            if (epoch+1) % 100 == 0:
                print(
                    'epoch: %d, Loss: %.3e' % 
                    (
                        epoch+1, 
                        loss.item()
                    )
                )
                
            
        # ues L-BFGS to optimize in the last
        # self.optimizer.step(self.loss_func)
        
        def predict(self, X):
            x = torch.tensor(X, requires_grad=True).float().to(device)

            self.dnn.eval()
            u = self.net_u(x, t) # calculate us
            f = self.net_f(x, t) # calculate f
            u = u.detach().cpu().numpy()  # turn u to NumPy
            f = f.detach().cpu().numpy()  # turn f to NumPy
            return u, f

In [3]:
# 13/12/2023
# generate training points
X = np.linspace(-1,1,100)
X = torch.tensor(X,stop_gradient=False,dtype='float32').to(device)
X = X.reshape([100,1])
X_bc = np.linspace(-1,1,2)
X_bc = torch.tensor(X_bc,stop_gradient=True,dtype='float32')
X_bc = X_bc.reshape([2,1])
Y_bc = torch.zeros([2,1],dtype='float32')

NameError: name 'np' is not defined

# 测试